In [9]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [11]:
# =======================
# 0. Definir ruta base del proyecto
# =======================
ruta_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
ruta_gold = os.path.join(ruta_base, "GOLD")

In [17]:
# --- 1. Cargar el dataset con características desde la capa GOLD ---
df_modelo = pd.read_parquet(os.path.join(ruta_gold, "maloob_features_for_training.parquet"))


In [21]:
# --- 2. Definir Características (X) y Objetivo (y) ---
features = ['produccion_mes_anterior', 'produccion_promedio_3m', 'mes', 'trimestre']
target = 'petroleo_mbd'

X = df_modelo[features]
y = df_modelo[target]

In [23]:
# --- 3. Dividir los datos en Entrenamiento y Prueba ---
# Usaremos el 80% de los datos para entrenar y el 20% más reciente para probar.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"\nDatos divididos: {len(X_train)} filas para entrenamiento, {len(X_test)} filas para prueba.")  


Datos divididos: 85 filas para entrenamiento, 22 filas para prueba.


In [27]:
# --- 4. Entrenar el Modelo XGBoost ---
model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        early_stopping_rounds=5 # Detiene el entrenamiento si el modelo no mejora
    )

In [29]:
# Entrenar el modelo usando los datos de prueba para la validación
model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False # No muestra todos los pasos del entrenamiento en pantalla
    )
print("Modelo entrenado exitosamente.")

Modelo entrenado exitosamente.


In [31]:
# --- 5. Evaluar el Modelo ---
# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

In [33]:
 # Calcular el error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
print("\nEvaluación del modelo completada.")
print(f"Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Esto significa que, en promedio, las predicciones del modelo se equivocan en {rmse:.2f} mil barriles diarios.")


Evaluación del modelo completada.
Error Cuadrático Medio (RMSE): 4.77
Esto significa que, en promedio, las predicciones del modelo se equivocan en 4.77 mil barriles diarios.
